### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

df = pd.read_csv("mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
#One-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

#Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


In [3]:
pwd

'C:\\Users\\eliza\\ICT\\ML\\4'

## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [7]:
#БИБЛИОТЕКИ
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.stats import mode

#Классификатор
class MyKNNClassifier:
    """
    Классификатор методом k-ближайших соседей.
    Оптимизирован для работы с большими данными, чтобы не падать с MemoryError.
    """
    def __init__(self, k=5, metric='euclidean', weights='uniform'):
        self.k = k
        self.metric = metric
        self.weights = weights

    def fit(self, X_train, y_train):
        """
        "Обучение" модели. Просто запоминаем обучающие данные.
        """
        self._train_data = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
        self._train_labels = y_train.values if isinstance(y_train, pd.Series) else y_train
        self._classes = np.unique(self._train_labels)

    def predict(self, X_test, batch_size=1000):
        """
        Предсказание меток для новых данных с использованием батчей для экономии памяти.
        """
        X_test_np = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
        num_test_samples = X_test_np.shape[0]
        
        all_predictions = []
        
        #Проходим по тестовым данным порциями (батчами)
        for i in range(0, num_test_samples, batch_size):
            X_batch = X_test_np[i : i + batch_size]
            
            #Считаем расстояния только для текущего батча
            distances = cdist(X_batch, self._train_data, metric=self.metric)
            
            k_nearest_indices = np.argsort(distances, axis=1)[:, :self.k]
            k_nearest_labels = self._train_labels[k_nearest_indices]

            if self.weights == 'uniform':
                batch_preds, _ = mode(k_nearest_labels, axis=1, keepdims=False)
                all_predictions.append(batch_preds)
                
            elif self.weights == 'distance':
                k_nearest_distances = np.take_along_axis(distances, k_nearest_indices, axis=1)
                epsilon = 1e-6
                vote_weights = 1 / (k_nearest_distances + epsilon)
                
                batch_preds = []
                for obj_idx in range(X_batch.shape[0]):
                    class_scores = {cls: 0 for cls in self._classes}
                    for neighbor_idx in range(self.k):
                        neighbor_label = k_nearest_labels[obj_idx, neighbor_idx]
                        neighbor_weight = vote_weights[obj_idx, neighbor_idx]
                        class_scores[neighbor_label] += neighbor_weight
                    
                    best_class = max(class_scores, key=class_scores.get)
                    batch_preds.append(best_class)
                
                all_predictions.append(np.array(batch_preds))
            else:
                raise ValueError("Неизвестный тип весов. Используйте 'uniform' или 'distance'.")
        
        return np.concatenate(all_predictions)

#Кросс-валидация
def perform_cross_validation(X, y, k_range, cv_folds=5):
    """
    Проводит кросс-валидацию для нашего KNN, чтобы найти лучший k.
    """
    print(f"Запускаем кросс-валидацию на {cv_folds} фолдах...")
    
    indices = np.arange(X.shape[0])
    np.random.seed(42)
    np.random.shuffle(indices)
    
    fold_indices = np.array_split(indices, cv_folds)
    
    k_scores = {k: [] for k in k_range}
    
    for i in range(cv_folds):
        print(f"  Фолд {i+1}/{cv_folds}...")
        val_idx = fold_indices[i]
        train_idx = np.concatenate([fold_indices[j] for j in range(cv_folds) if j != i])
        
        X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
        X_val_fold, y_val_fold = X.iloc[val_idx], y.iloc[val_idx]

        for k in k_range:
            model = MyKNNClassifier(k=k, weights='distance') # Можно поменять на 'uniform'
            model.fit(X_train_fold, y_train_fold)
            
            # Важно: здесь вызывается наш новый `predict` с батчами
            y_pred = model.predict(X_val_fold) 
            
            accuracy = np.mean(y_pred == y_val_fold.values)
            k_scores[k].append(accuracy)

    mean_k_scores = {k: np.mean(scores) for k, scores in k_scores.items()}
    return mean_k_scores


df = pd.read_csv('mlbootcamp5_train.csv', sep=';') 

try:
    #Пытаемся загрузить реальный датасет
    df = pd.read_csv('mlbootcamp5_train.csv', sep=';')
    df = df.drop(columns=['id']) # Удаляем ненужный столбец id
    # Возьмем только часть данных, чтобы кросс-валидация не шла вечность
    df = df.sample(n=10000, random_state=42) 
except FileNotFoundError:
    print("Файл 'mlbootcamp5_train.csv' не найден. Создаем фейковые данные для демонстрации.")
    from sklearn.datasets import make_classification
    X_fake, y_fake = make_classification(n_samples=5000, n_features=11, n_informative=8, n_redundant=0, n_classes=2, random_state=42)
    features_list = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
                     'cholesterol', 'gluc', 'smoke', 'alco', 'active']
    df = pd.DataFrame(X_fake, columns=features_list)
    df['cardio'] = y_fake


#Выбираем признаки и целевой столбец
features = df.columns.drop('cardio').tolist()
target = 'cardio'

X_data = df[features]
y_data = df[target]

#Нормализуем данные
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_data_scaled = pd.DataFrame(scaler.fit_transform(X_data), columns=features)


#Задаем диапазон k, который хотим проверить
k_values_to_test = list(range(5, 26, 4)) # Проверим k = 5, 9, 13, 17, 21, 25

#Запускаем нашу кросс-валидацию
#Используем нормализованные данные X_data_scaled
cv_scores = perform_cross_validation(X_data_scaled, y_data, k_values_to_test, cv_folds=4) # 4 фолда, чтобы было быстрее

print("\nРезультаты кросс-валидации (k: средняя точность):")
for k, score in cv_scores.items():
    print(f"k={k}: {score:.4f}")

#Находим k с наилучшим результатом
best_k_found = max(cv_scores, key=cv_scores.get)
print(f"\n---> Нашли лучший параметр k: {best_k_found} с точностью {cv_scores[best_k_found]:.4f}")

#Обучаем финальную модель на всех данных с лучшим k
print(f"\nОбучаем итоговую модель с k={best_k_found}...")
final_knn_model = MyKNNClassifier(k=best_k_found, weights='distance')
#Обучаем на полных нормализованных данных
final_knn_model.fit(X_data_scaled, y_data)

Запускаем кросс-валидацию на 4 фолдах...
  Фолд 1/4...
  Фолд 2/4...
  Фолд 3/4...
  Фолд 4/4...

Результаты кросс-валидации (k: средняя точность):
k=5: 0.6334
k=9: 0.6401
k=13: 0.6477
k=17: 0.6487
k=21: 0.6559
k=25: 0.6535

---> Нашли лучший параметр k: 21 с точностью 0.6559

Обучаем итоговую модель с k=21...


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

#Подготовка данных

# Предположим, что df — ваш DataFrame
features = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
target = 'cardio'

X = df[features]
y = df[target]

# Разбиваем на тренировочную и тестовую выборки (например, 70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#Обучение моделей

# kNN (например, с k=5)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Наивный Байес
nb = GaussianNB()
nb.fit(X_train, y_train)

# Решающее дерево
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

#Предсказания и оценка

models = {
    'kNN': knn,
    'Naive Bayes': nb,
    'Decision Tree': dt
}

results = {}

for name, model in models.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # Вероятности положительного класса

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    loss = log_loss(y_test, y_proba)

    results[name] = {
        'Accuracy': acc,
        'F1-score': f1,
        'ROC AUC': roc_auc,
        'Log Loss': loss
    }

#Вывод результатов и определение лучших

df_results = pd.DataFrame(results).T
print(df_results)

# Определение лучших по каждой метрике
best_metrics = df_results.idxmax()
print("\nЛучший классификатор по каждой метрике:")
print(best_metrics)

               Accuracy  F1-score   ROC AUC   Log Loss
kNN            0.679143  0.672149  0.723429   2.365549
Naive Bayes    0.541238  0.213551  0.667530   0.785127
Decision Tree  0.617619  0.617619  0.617655  13.769303

Лучший классификатор по каждой метрике:
Accuracy              kNN
F1-score              kNN
ROC AUC               kNN
Log Loss    Decision Tree
dtype: object


**Комментарии:** Ваши комментарии здесь.